In [1]:
import pandas as pd

In [2]:
pd.__version__

'0.19.0'

## We start basic analysis from here.

In [7]:
import sys
import operator

pd.set_option('display.max_colwidth', sys.maxsize)

chunk_size = 100000
reader = pd.read_table('../../dataset/academic/yelp_academic_dataset_review.json', header=None, chunksize=chunk_size)

count_of_stars = {}
num_reviews_of_business = {}
active_user_of_business = {}
votes_of_business = {}

chunk_index = 0
for chunks in reader:
    
    start_index = chunk_size * chunk_index
    for index in range(start_index, start_index + len(chunks)):
        data_json = pd.read_json(chunks[0][index], typ='series')

        stars = data_json['stars']
        if (stars not in count_of_stars):
            count_of_stars[stars] = 1
        else: 
            count_of_stars[stars] = count_of_stars[stars] + 1

        business_id = data_json['business_id']
        if (business_id not in num_reviews_of_business):
            num_reviews_of_business[business_id] = 1
        else: 
            num_reviews_of_business[business_id] = num_reviews_of_business[business_id] + 1

        user_id = data_json['user_id']
        user_business_key = business_id + '_' + user_id
        if (user_business_key not in active_user_of_business):
            active_user_of_business[user_business_key] = 1
        else: 
            active_user_of_business[user_business_key] = active_user_of_business[user_business_key] + 1
            
        votes_json = data_json['votes']
        num_votes = votes_json['funny'] + votes_json['useful'] + votes_json['cool']
        if (business_id not in votes_of_business):
            votes_of_business[business_id] = num_votes
        else: 
            votes_of_business[business_id] = votes_of_business[business_id] + num_votes
        
    
    chunk_index += 1

#     if (chunk_index > 2):
#         break

print("\nTotal count of each different stars:")
print(count_of_stars)

print("\nSort business by number of reviews:")
sorted_num_reviews_of_business = sorted(num_reviews_of_business.items(), key=operator.itemgetter(1), reverse=True)
print(sorted_num_reviews_of_business[:10])

print("\nSort users by number of reviews on the same business:")
sorted_active_user_of_business = sorted(active_user_of_business.items(), key=operator.itemgetter(1), reverse=True)
print(sorted_active_user_of_business[:10])

print("\nSort business by number of votes from reviews:")
sorted_votes_of_business = sorted(votes_of_business.items(), key=operator.itemgetter(1), reverse=True)
print(sorted_votes_of_business[:10])


Total count of each different stars:
{1: 331786, 2: 224334, 3: 321700, 4: 674636, 5: 1132610}

Sort business by number of reviews:
[('4bEjOyTaDG24SY5TxsaUNQ', 5558), ('sIyHTizqAiGu12XMLX3N3g', 4531), ('zt1TpTuJ6y9n551sw9TaEg', 4333), ('aGbjLWzcrnEx2ZmMCFm3EA', 4149), ('2e2e7WgqU1BnpxmQL5jbfw', 4103), ('iUPJmJvHy9fVfRxsuwwdLQ', 3584), ('Xhg93cMdemu5pAMkDoEdtQ', 3330), ('QbmcCE_cLq4WO8ZMKImaLw', 3202), ('YNQgak-ZLtYJQxlDwN-qIg', 3107), ('lliksv-tglfUz1T3B3vgvA', 2911)]

Sort users by number of reviews on the same business:
[('T1jYZFB_7cqdhuvzpxfFWQ_q200TCFOheJfS_MvvwMfJw', 31), ('6LoNRotewF81fxka6L_8Jw_q200TCFOheJfS_MvvwMfJw', 19), ('yumgsa9m02CACzq0dHZP7Q_q200TCFOheJfS_MvvwMfJw', 18), ('8aPTQeDYOMNEu907WfflCQ__Q4Ks28-Jks-TjFT-YXw_A', 18), ('DzL4cUt5AVYRzzT7DHOgxg_GfJBijGnGRIbXRkNMzS6Eg', 16), ('6O9iAmAv7vUl-Eeqb9bmpw_C2RiYrCAdvj7Sh0p6I8G7Q', 16), ('iaEqryy7OwAOAQkLWDBquQ_kG8jkVFgOAOq6Ht6RvAjEQ', 15), ('RZTcKugP9dGWFkO4RpbVEQ_FbdB9KXyuvT8rC_zmqRSuQ', 15), ('zsAlOIoPCdVZO2kVAb5YJw_pk6ci-

## Calculate the frequency of working_type

In [144]:
df_business = pd.read_json('../../dataset/academic/yelp_academic_dataset_business.json', lines=True)
# df_business = df_business[df_business['open'] == True]

df_business_restaurants = df_business[df_business['categories'].apply(lambda x: 'Restaurants' in x)].reset_index(drop=True)[['business_id', 'stars', 'hours']]
df_business_restaurants['stars_round'] = df_business_restaurants['stars'].apply(round)

df_business_restaurants.head()

,business_id,stars,hours,stars_round
0,5UmKMjUEUNdYWqANhGckJw,3.5,"{'Thursday': {'close': '21:00', 'open': '11:00...",4
1,mVHrayjG3uZ_RLHkLj-AMg,4.5,"{'Thursday': {'close': '19:00', 'open': '10:00...",4
2,KayYbHCt-RkbGcPdGOThNg,4.0,"{'Tuesday': {'close': '02:00', 'open': '11:00'...",4
3,wJr6kSA5dchdgOdwH6dZ2w,3.5,"{'Tuesday': {'close': '02:00', 'open': '08:00'...",4
4,fNGIbpazjTRdXgwRY_NIXA,4.0,"{'Tuesday': {'close': '23:00', 'open': '11:00'...",4


In [306]:
from datetime import time

group_by_stars = df_business_restaurants.groupby('stars_round')

business_of_stars = {}
for star in group_by_stars.groups:
    group = group_by_stars.get_group(star)
    business_of_stars[star] = group.assign(working_type=lambda x: x['hours'])

WORKING_TYPES = {
    "WEEKEND_TYPE": "weekend",
    "BREAKFAST_TYPE": "breakfast",
    "LUNCH_TYPE": "lunch",
    "AFTER_LUNCH_TYPE": "after-lunch",
    "DINNER_TYPE": "dinner",
    "NIGHT_TYPE": "night",
}

breakfast = time(8)
lunch = time(12)
after_lunch = time(15)
dinner = time(18)
night = time(0)

def in_between(start, end, check):
    if start == end: # 24 hours
        return True
    if start < end:
        return start <= check < end
    else: # over midnight e.g., 23:30-04:15
        return start <= check or check < end

TYPE_THRESHOLD = 1
def spec_hours_to_type(s):
    types = []
    
    breakfast_count = 0
    lunch_count = 0
    after_lunch_count = 0
    dinner_count = 0
    night_count = 0

    for day in s:
        clo = s[day]['close']
        op = s[day]['open']

        h, m = clo.split(':')
        clo_t = time(int(h), int(m))

        h, m = op.split(':')
        op_t = time(int(h), int(m))
        
        if in_between(op_t, clo_t, breakfast):
            breakfast_count += 1
        
        if in_between(op_t, clo_t, lunch):
            lunch_count += 1
        
        if in_between(op_t, clo_t, after_lunch):
            after_lunch_count += 1
        
        if in_between(op_t, clo_t, dinner):
            dinner_count += 1
        
        if in_between(op_t, clo_t, night):
            night_count += 1
        
        if (day in ['Saturday', 'Sunday']) and (WORKING_TYPES["WEEKEND_TYPE"] not in types):
            types.append(WORKING_TYPES["WEEKEND_TYPE"])
            
    if breakfast_count >= TYPE_THRESHOLD:
        types.append(WORKING_TYPES["BREAKFAST_TYPE"])
        
    if lunch_count >= TYPE_THRESHOLD:
        types.append(WORKING_TYPES["LUNCH_TYPE"])
    
    if after_lunch_count >= TYPE_THRESHOLD:
        types.append(WORKING_TYPES["AFTER_LUNCH_TYPE"])
    
    if dinner_count >= TYPE_THRESHOLD:
        types.append(WORKING_TYPES["DINNER_TYPE"])
        
    if night_count >= TYPE_THRESHOLD:
        types.append(WORKING_TYPES["NIGHT_TYPE"])
    
    return join_types(types)

def hours_to_type(s):
    if s:
        return spec_hours_to_type(s)
    else:
        return join_types(WORKING_TYPES.values())

def join_types(ts):
    # reorder
    ordered_types = []
    for t in WORKING_TYPES.values():
        if t in ts:
            ordered_types.append(t)
    return '_'.join(ordered_types)

types_freq_of_stars = {}
num_working_types = 2**len(WORKING_TYPES) - 1

for star in business_of_stars:
    business_of_star_df = business_of_stars[star]
    business_of_star_df['working_type'] = business_of_star_df['working_type'].apply(hours_to_type)
    
    # count frequency of different type
    types_freq = {}
    num_business = len(business_of_star_df)
    
    working_type_of_business = business_of_star_df.groupby('working_type')
    for wt in working_type_of_business.groups:
        # we use the add-one or Laplace smoothing
        types_freq[wt] = (len(working_type_of_business.get_group(wt)) + 1.0) / (num_business + num_working_types)
    
    types_freq['default'] = 1.0 / (num_business + num_working_types)
    types_freq_of_stars[star] = types_freq


types_freq_of_stars_df = pd.DataFrame.from_dict(types_freq_of_stars)

for c in types_freq_of_stars_df.columns.values:
    types_freq_of_stars_df[c] = types_freq_of_stars_df[c].fillna(types_freq_of_stars_df[c]['default'])

print(types_freq_of_stars_df)
business_of_stars[3].head()

                                                         1         2  \
after-lunch                                       0.006024  0.000225   
breakfast                                         0.006024  0.000225   
breakfast_weekend                                 0.012048  0.000225   
breakfast_weekend_dinner                          0.006024  0.000449   
default                                           0.006024  0.000225   
dinner                                            0.006024  0.000449   
lunch                                             0.006024  0.001123   
lunch_after-lunch                                 0.006024  0.001572   
lunch_breakfast                                   0.006024  0.001347   
lunch_breakfast_after-lunch                       0.006024  0.001572   
lunch_breakfast_dinner_after-lunch                0.006024  0.002246   
lunch_breakfast_weekend                           0.006024  0.004042   
lunch_breakfast_weekend_after-lunch               0.006024  0.00

,business_id,stars,hours,stars_round,working_type
9,rv7CY8G_XibTx82YhuqQRw,3.0,{},3,night_lunch_breakfast_weekend_dinner_after-lunch
19,W7ysp8GLCBrs0GXIVwAcBQ,3.0,{},3,night_lunch_breakfast_weekend_dinner_after-lunch
23,TfvWSIAMFZ9zI3y2-K6wOA,3.0,"{'Tuesday': {'close': '22:00', 'open': '11:00'...",3,lunch_weekend_dinner_after-lunch
25,6p9TlP2I8S4agxYMNFSDjg,3.0,"{'Tuesday': {'close': '23:00', 'open': '11:00'...",3,lunch_weekend_dinner_after-lunch
26,JDDeaNfb0JXD1NbznSIC9g,3.0,"{'Tuesday': {'close': '22:00', 'open': '11:00'...",3,lunch_weekend_dinner_after-lunch
